# InstructABSA - Reproduction and Improvement

This notebook reproduces and improves results from the paper:
**"InstructABSA: Instruction Learning for Aspect Based Sentiment Analysis" (NAACL 2024)**

## Contents
1. Setup & Dependencies
2. Reproduce Paper Results (ATSC Task)
3. Baseline Comparison
4. Prompt Engineering (4 vs 8 examples)
5. Flan-T5 vs Tk-Instruct Comparison
6. Persian Cross-lingual Test
7. Results Summary & Visualization

## 1. Setup & Dependencies

In [ ]:
# Install required packages (run once)
# !pip install torch transformers pandas scikit-learn tqdm matplotlib

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm import tqdm
import random
import matplotlib.pyplot as plt

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

## 2. Reproduce Paper Results (ATSC Task)

**Goal:** Reproduce Table 2 results from the paper on SemEval15 Restaurants dataset.

**Expected Result:** Accuracy ≈ 84.50%

In [ ]:
# Load model
model_name = "kevinscaria/atsc_tk-instruct-base-def-pos-neg-neut-combined"
print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
model.eval()
print("Model loaded!")

In [ ]:
# Load dataset
# Dataset source: https://github.com/kevinscaria/InstructABSA/tree/main/Dataset
df = pd.read_csv("Dataset/SemEval15/Test/Restaurants_Test.csv")
print(f"Loaded {len(df)} samples")
df.head()

In [ ]:
# Instruction template
instruction = """Definition: The output will be 'positive', 'negative', or 'neutral' based on the sentiment of the aspect.

Now complete the following example-
input: {text} The aspect is {aspect}.
output:"""

# Evaluation
correct = 0
total = 0

for _, row in tqdm(df.iterrows(), total=len(df)):
    text = row['raw_text']
    aspects = eval(row['aspectTerms'])
    
    for asp in aspects:
        term = asp['term']
        true_polarity = asp['polarity'].lower()
        
        if term == 'noaspectterm' or true_polarity == 'none':
            continue
        
        prompt = instruction.format(text=text, aspect=term)
        inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
        
        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=10)
        
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().lower()
        
        if pred == true_polarity:
            correct += 1
        total += 1

reproduction_acc = correct / total * 100
print(f"\n{'='*40}")
print(f"Results on SemEval15 Restaurants Test")
print(f"{'='*40}")
print(f"Correct: {correct}/{total}")
print(f"Accuracy: {reproduction_acc:.2f}%")
print(f"\nPaper reports: 84.50%")

## 3. Baseline Comparison

Compare InstructABSA with simple baselines:
- **Random Baseline:** Randomly predict positive/negative/neutral (~33%)
- **Majority Baseline:** Always predict the most common class (~60%)

In [ ]:
# Extract samples
samples = []
for _, row in df.iterrows():
    text = row['raw_text']
    aspects = eval(row['aspectTerms'])
    for asp in aspects:
        term = asp['term']
        polarity = asp['polarity'].lower()
        if term != 'noaspectterm' and polarity != 'none':
            samples.append({'text': text, 'aspect': term, 'polarity': polarity})

print(f"Total samples: {len(samples)}")

# Class distribution
pos_count = sum(1 for s in samples if s['polarity'] == 'positive')
neg_count = sum(1 for s in samples if s['polarity'] == 'negative')
neu_count = sum(1 for s in samples if s['polarity'] == 'neutral')

print(f"\nClass distribution:")
print(f"  Positive: {pos_count} ({pos_count/len(samples)*100:.1f}%)")
print(f"  Negative: {neg_count} ({neg_count/len(samples)*100:.1f}%)")
print(f"  Neutral:  {neu_count} ({neu_count/len(samples)*100:.1f}%)")

In [ ]:
# Random Baseline
random.seed(42)
random_correct = sum(1 for s in samples if random.choice(['positive', 'negative', 'neutral']) == s['polarity'])
random_acc = random_correct / len(samples) * 100
print(f"Random Baseline Accuracy: {random_acc:.2f}%")

# Majority Baseline (always predict 'positive')
majority_correct = sum(1 for s in samples if s['polarity'] == 'positive')
majority_acc = majority_correct / len(samples) * 100
print(f"Majority Baseline Accuracy: {majority_acc:.2f}%")

# InstructABSA
print(f"InstructABSA Accuracy: {reproduction_acc:.2f}%")

In [ ]:
# Visualization: Baseline Comparison
methods = ['Random\nBaseline', 'Majority\nBaseline', 'InstructABSA']
accuracies = [random_acc, majority_acc, reproduction_acc]
colors = ['#e74c3c', '#f39c12', '#2ecc71']

plt.figure(figsize=(8, 5))
bars = plt.bar(methods, accuracies, color=colors, edgecolor='black', linewidth=1.2)

for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{acc:.2f}%', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.ylabel('Accuracy (%)', fontsize=12)
plt.title('Baseline Comparison - ATSC Task (Rest15)', fontsize=14, fontweight='bold')
plt.ylim(0, 100)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('Output/baseline_comparison.png', dpi=150)
plt.show()

## 4. Prompt Engineering (4 vs 8 examples)

**Question:** Does adding more examples in the prompt improve accuracy?

- Paper uses 6 examples (2 positive + 2 negative + 2 neutral)
- We test: 4 examples and 8 examples

In [ ]:
# Prompt with 4 examples
prompt_4_examples = """Definition: The output will be 'positive', 'negative', or 'neutral' based on the sentiment of the aspect.

Example 1-
input: The food was delicious. The aspect is food.
output: positive

Example 2-
input: The service was terrible. The aspect is service.
output: negative

Example 3-
input: The price is reasonable. The aspect is price.
output: neutral

Example 4-
input: I loved the atmosphere. The aspect is atmosphere.
output: positive

Now complete the following example-
input: {text} The aspect is {aspect}.
output:"""

# Prompt with 8 examples
prompt_8_examples = """Definition: The output will be 'positive', 'negative', or 'neutral' based on the sentiment of the aspect.

Example 1-
input: The food was delicious. The aspect is food.
output: positive

Example 2-
input: I loved the atmosphere. The aspect is atmosphere.
output: positive

Example 3-
input: Great service and friendly staff. The aspect is service.
output: positive

Example 4-
input: The service was terrible. The aspect is service.
output: negative

Example 5-
input: The wait time was too long. The aspect is wait time.
output: negative

Example 6-
input: Overpriced for what you get. The aspect is price.
output: negative

Example 7-
input: The restaurant is located downtown. The aspect is restaurant.
output: neutral

Example 8-
input: They serve Italian food. The aspect is food.
output: neutral

Now complete the following example-
input: {text} The aspect is {aspect}.
output:"""

In [ ]:
def evaluate_prompt(prompt_template, samples):
    correct = 0
    for s in tqdm(samples):
        prompt = prompt_template.format(text=s['text'], aspect=s['aspect'])
        inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
        
        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=10)
        
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().lower()
        
        if pred == s['polarity']:
            correct += 1
    
    return correct / len(samples) * 100

# Test 4 examples
print("Testing 4 examples...")
acc_4 = evaluate_prompt(prompt_4_examples, samples)
print(f"4 Examples Accuracy: {acc_4:.2f}%")

# Test 8 examples
print("\nTesting 8 examples...")
acc_8 = evaluate_prompt(prompt_8_examples, samples)
print(f"8 Examples Accuracy: {acc_8:.2f}%")

In [ ]:
# Visualization: Prompt Engineering Results
prompts_names = ['Paper\n(6 examples)', '4 Examples', '8 Examples']
prompts_acc = [84.50, acc_4, acc_8]
colors = ['#3498db', '#2ecc71', '#9b59b6']

plt.figure(figsize=(8, 5))
bars = plt.bar(prompts_names, prompts_acc, color=colors, edgecolor='black', linewidth=1.2)

for bar, acc in zip(bars, prompts_acc):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, 
             f'{acc:.2f}%', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.axhline(y=84.50, color='red', linestyle='--', linewidth=2, label='Paper Baseline')
plt.ylabel('Accuracy (%)', fontsize=12)
plt.title('Prompt Engineering - Number of Examples', fontsize=14, fontweight='bold')
plt.ylim(80, 90)
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('Output/prompt_engineering.png', dpi=150)
plt.show()

## 5. Flan-T5 vs Tk-Instruct Comparison

**Question:** Can a general-purpose model (Flan-T5) compete with task-specific model (InstructABSA)?

| Model | Training Tasks | Fine-tuned on ABSA? |
|-------|----------------|---------------------|
| Tk-Instruct | 1600 tasks | Yes (InstructABSA) |
| Flan-T5 | 1800 tasks | No |

In [ ]:
# Load Flan-T5
print("Loading Flan-T5-base...")
flan_model_name = "google/flan-t5-base"
flan_tokenizer = AutoTokenizer.from_pretrained(flan_model_name)
flan_model = AutoModelForSeq2SeqLM.from_pretrained(flan_model_name).to(device)
flan_model.eval()
print("Flan-T5 loaded!")

In [ ]:
# Flan-T5 Prompts
flan_zero_shot = """What is the sentiment toward the aspect in the review? Answer with one word: positive, negative, or neutral.

Review: {text}. Aspect: {aspect}. Sentiment:"""

flan_6_shot = """What is the sentiment toward the aspect in the review? Answer with: positive, negative, or neutral.

Example 1:
Review: The food was delicious. Aspect: food. Sentiment: positive

Example 2:
Review: I loved the atmosphere. Aspect: atmosphere. Sentiment: positive

Example 3:
Review: The service was terrible. Aspect: service. Sentiment: negative

Example 4:
Review: The wait time was too long. Aspect: wait time. Sentiment: negative

Example 5:
Review: The price is reasonable. Aspect: price. Sentiment: neutral

Example 6:
Review: They serve Italian food. Aspect: food. Sentiment: neutral

Now answer:
Review: {text}. Aspect: {aspect}. Sentiment:"""

In [ ]:
def evaluate_flan(prompt_template, samples):
    correct = 0
    for s in tqdm(samples):
        prompt = prompt_template.format(text=s['text'], aspect=s['aspect'])
        inputs = flan_tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
        
        with torch.no_grad():
            outputs = flan_model.generate(**inputs, max_length=10)
        
        pred = flan_tokenizer.decode(outputs[0], skip_special_tokens=True).strip().lower()
        
        if 'positive' in pred:
            pred_label = 'positive'
        elif 'negative' in pred:
            pred_label = 'negative'
        elif 'neutral' in pred:
            pred_label = 'neutral'
        else:
            pred_label = pred
        
        if pred_label == s['polarity']:
            correct += 1
    
    return correct / len(samples) * 100

# Test Flan-T5 zero-shot
print("Testing Flan-T5 zero-shot...")
flan_zero_acc = evaluate_flan(flan_zero_shot, samples)
print(f"Flan-T5 (zero-shot): {flan_zero_acc:.2f}%")

# Test Flan-T5 6-shot
print("\nTesting Flan-T5 6-shot...")
flan_6_acc = evaluate_flan(flan_6_shot, samples)
print(f"Flan-T5 (6-shot): {flan_6_acc:.2f}%")

In [ ]:
# Visualization: Flan-T5 vs InstructABSA
models_names = ['Flan-T5\n(zero-shot)', 'Flan-T5\n(6-shot)', 'InstructABSA\n(fine-tuned)']
model_accs = [flan_zero_acc, flan_6_acc, reproduction_acc]
colors = ['#3498db', '#2980b9', '#2ecc71']

plt.figure(figsize=(8, 5))
bars = plt.bar(models_names, model_accs, color=colors, edgecolor='black', linewidth=1.2)

for bar, acc in zip(bars, model_accs):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, 
             f'{acc:.2f}%', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.ylabel('Accuracy (%)', fontsize=12)
plt.title('Flan-T5 vs InstructABSA - ATSC Task', fontsize=14, fontweight='bold')
plt.ylim(75, 90)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('Output/flan_vs_instructabsa.png', dpi=150)
plt.show()

## 6. Persian Cross-lingual Test

**Question:** Can an English-only model understand Persian sentiment?

We test on 15 Persian sentences with zero-shot and 6-shot Persian examples.

In [ ]:
# Persian test data
persian_test = [
    {"text": "غذا عالی بود", "aspect": "غذا", "polarity": "positive"},
    {"text": "پیتزا خوشمزه بود", "aspect": "پیتزا", "polarity": "positive"},
    {"text": "سرویس افتضاح بود", "aspect": "سرویس", "polarity": "negative"},
    {"text": "گارسون بی ادب بود", "aspect": "گارسون", "polarity": "negative"},
    {"text": "قیمت معمولی بود", "aspect": "قیمت", "polarity": "neutral"},
    {"text": "رستوران شلوغ بود", "aspect": "رستوران", "polarity": "neutral"},
    {"text": "کیفیت غذا بد بود", "aspect": "کیفیت غذا", "polarity": "negative"},
    {"text": "فضای رستوران دلنشین بود", "aspect": "فضا", "polarity": "positive"},
    {"text": "منو متنوع بود", "aspect": "منو", "polarity": "positive"},
    {"text": "انتظار زیادی کشیدیم", "aspect": "انتظار", "polarity": "negative"},
    {"text": "پارکینگ داشت", "aspect": "پارکینگ", "polarity": "neutral"},
    {"text": "دسر عالی بود", "aspect": "دسر", "polarity": "positive"},
    {"text": "نوشیدنی سرد نبود", "aspect": "نوشیدنی", "polarity": "negative"},
    {"text": "میز تمیز بود", "aspect": "میز", "polarity": "positive"},
    {"text": "صندلی راحت نبود", "aspect": "صندلی", "polarity": "negative"},
]

print(f"Persian test samples: {len(persian_test)}")

In [ ]:
# Persian prompts
persian_zero_shot = """Definition: The output will be 'positive', 'negative', or 'neutral' based on the sentiment of the aspect.

Now complete the following example-
input: {text} The aspect is {aspect}.
output:"""

persian_6_shot = """Definition: The output will be 'positive', 'negative', or 'neutral' based on the sentiment of the aspect.

Example 1-
input: غذا خوشمزه بود. The aspect is غذا.
output: positive

Example 2-
input: فضای رستوران عالی بود. The aspect is فضا.
output: positive

Example 3-
input: سرویس بد بود. The aspect is سرویس.
output: negative

Example 4-
input: زمان انتظار طولانی بود. The aspect is انتظار.
output: negative

Example 5-
input: قیمت مناسب بود. The aspect is قیمت.
output: neutral

Example 6-
input: رستوران در مرکز شهر بود. The aspect is رستوران.
output: neutral

Now complete the following example-
input: {text} The aspect is {aspect}.
output:"""

In [ ]:
def evaluate_persian(prompt_template):
    correct = 0
    for item in persian_test:
        prompt = prompt_template.format(text=item['text'], aspect=item['aspect'])
        inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
        
        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=10)
        
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().lower()
        
        status = "✅" if pred == item['polarity'] else "❌"
        print(f"{status} {item['text']:<25} | True: {item['polarity']:<10} Pred: {pred}")
        
        if pred == item['polarity']:
            correct += 1
    
    return correct / len(persian_test) * 100

# Test Persian zero-shot
print("=" * 60)
print("Testing: Zero-shot")
print("=" * 60)
persian_zero_acc = evaluate_persian(persian_zero_shot)
print(f"\n→ Accuracy: {persian_zero_acc:.2f}%")

# Test Persian 6-shot
print("\n" + "=" * 60)
print("Testing: 6-shot Persian")
print("=" * 60)
persian_6_acc = evaluate_persian(persian_6_shot)
print(f"\n→ Accuracy: {persian_6_acc:.2f}%")

In [ ]:
# Visualization: Persian Results
persian_methods = ['Random\nBaseline', 'Zero-shot', '6-shot Persian']
persian_accs = [33.33, persian_zero_acc, persian_6_acc]
colors = ['#e74c3c', '#3498db', '#9b59b6']

plt.figure(figsize=(8, 5))
bars = plt.bar(persian_methods, persian_accs, color=colors, edgecolor='black', linewidth=1.2)

for bar, acc in zip(bars, persian_accs):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{acc:.2f}%', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.ylabel('Accuracy (%)', fontsize=12)
plt.title('Persian Cross-lingual Test', fontsize=14, fontweight='bold')
plt.ylim(0, 60)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('Output/persian_results.png', dpi=150)
plt.show()

print("\n⚠️ Finding: The English-only model cannot understand Persian.")
print("   Adding Persian examples makes it worse (model gets confused)!")

## 7. Results Summary

### Key Findings

| Experiment | Result | Note |
|------------|--------|------|
| Paper Reproduction | 84.50% | ✓ Matched |
| 4 Examples | +1.48% | Improvement |
| 8 Examples | +1.29% | Improvement |
| Flan-T5 (zero-shot) | 84.69% | Comparable! |
| Persian Zero-shot | ~40% | Random level |
| Persian 6-shot | ~20% | Worse! |

### Conclusions

1. **Reproduction:** Successfully reproduced paper results (84.50%)
2. **Prompt Engineering:** More examples can help, but trade-off with inference speed
3. **Flan-T5:** General-purpose model performs comparably without task-specific fine-tuning
4. **Persian:** Model needs fine-tuning for cross-lingual transfer; adding Persian examples confuses the model

In [ ]:
# Final Summary Chart
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Chart 1: Model Comparison
all_methods = ['Random', 'Majority', 'InstructABSA', 'Flan-T5']
all_accs = [random_acc, majority_acc, reproduction_acc, flan_zero_acc]
colors1 = ['#e74c3c', '#f39c12', '#2ecc71', '#3498db']

bars1 = axes[0].bar(all_methods, all_accs, color=colors1, edgecolor='black')
for bar, acc in zip(bars1, all_accs):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                 f'{acc:.1f}%', ha='center', fontsize=10, fontweight='bold')
axes[0].set_ylabel('Accuracy (%)')
axes[0].set_title('Model Comparison (English)', fontweight='bold')
axes[0].set_ylim(0, 100)
axes[0].grid(axis='y', alpha=0.3)

# Chart 2: Prompt Engineering
prompt_methods = ['Paper (6)', '4 Examples', '8 Examples']
prompt_accs = [84.50, acc_4, acc_8]
colors2 = ['#3498db', '#2ecc71', '#9b59b6']

bars2 = axes[1].bar(prompt_methods, prompt_accs, color=colors2, edgecolor='black')
for bar, acc in zip(bars2, prompt_accs):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2, 
                 f'{acc:.2f}%', ha='center', fontsize=10, fontweight='bold')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Prompt Engineering Results', fontweight='bold')
axes[1].set_ylim(80, 90)
axes[1].axhline(y=84.50, color='red', linestyle='--', label='Paper baseline')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('Output/final_summary.png', dpi=150)
plt.show()

print("\n" + "="*60)
print("✅ All experiments completed!")
print("📁 Charts saved to Output/ folder")
print("="*60)